# Member Classification Model

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import sys
from torch_geometric.data import DataLoader, Dataset, Data
import lightning.pytorch as pl
import seaborn as sns
import pandas as pd
import os
from tqdm import tqdm
import torch
import itertools
import yaml
from pytorch_lightning.loggers import WandbLogger

import matplotlib.pyplot as plt
from torch_geometric.utils import to_scipy_sparse_matrix
import scipy.sparse as sps
import xxhash
from torch_cluster import knn

from epic_clustering.utils import plot_clusters, get_cluster_pos
from epic_clustering.models import MemberClassification
from epic_clustering.scoring import weighted_v_score

## 1. Training Loop

Training took me about 2 hours on a single (A100) GPU. If you use a smaller GPU, you may need to decrease the batch size. The configuration I used for this submission is:

```
input_dir: /global/cfs/cdirs/m3443/data/PowerWeek/train/train/
project: PowerWeek_MemberClassification
checkpoint_dir: /global/cfs/cdirs/m3443/data/PowerWeek/checkpoints/

data_split: [2000, 10, 10]
batch_size: 20
input_features: 12
emb_hidden: 1024
nb_layer: 4
activation: ReLU

warmup: 10
lr: 0.01
patience: 30
max_epochs: 10
factor: 0.7
num_seeds: 40
```

In [ ]:
with open("member_classification.yaml") as f:
    member_classification_config = yaml.safe_load(f)
model = MemberClassification(member_classification_config)
model.setup(stage="fit")

In [ ]:
logger = WandbLogger(project=member_classification_config["project"])
trainer = pl.Trainer(devices=1, accelerator="gpu", max_epochs=300, logger=logger)
trainer.fit(model)

In [ ]:
event = model.trainset[-1]

In [ ]:
event

In [ ]:
true_edges = event.edge_index[:, event.y.bool()]

In [ ]:
# Collect seeds in one tensor
seeds_idx = true_edges.unique()

In [ ]:
input_dir = "/global/cfs/cdirs/m3443/data/PowerWeek/train/train"
csv_file = os.path.join(input_dir, "/global/cfs/cdirs/m3443/data/PowerWeek/train/train/train_ePIC_event0_1000.csv")
events_df = pd.read_csv(csv_file)

In [ ]:
input_dir = "/global/cfs/cdirs/m3443/data/PowerWeek/train/train"
num_events = sum(model.hparams["data_split"])
csv_files = sorted([os.path.join(input_dir, f) for f in os.listdir(input_dir) if f.endswith('.csv')])[:num_events//1000 + 1]
events_df = pd.concat([pd.read_csv(f) for f in sorted(csv_files)])
if num_events is not None:
    events_df = events_df[events_df["event"].isin(sorted(events_df["event"].unique())[:num_events])]
events_df['clusterID'] = events_df['clusterID'].astype(np.uint64) # Needed for some reason?

In [ ]:
event_df = events_df[events_df.event == 10008]

In [ ]:
# Collect nonseeds in another tensor
nonseeds_idx = torch.from_numpy(event_df.hit_number[~np.isin(event_df.hit_number.values, seeds_idx.long().numpy())].values).unique()

In [ ]:
# For each nonseed find closest seed with knn=1
nonseeds_to_seeds = knn(torch.from_numpy(event_df[np.isin(event_df.hit_number.values, seeds_idx.long().numpy())][['posx', 'posy']].to_numpy()), torch.from_numpy(event_df[np.isin(event_df.hit_number.values, nonseeds_idx.long().numpy())][['posx', 'posy']].to_numpy()), 1)

# Convert 0, .., N indices back to original seed_idx and nonseed_idx
nonseeds_to_seeds = torch.stack([seeds_idx[nonseeds_to_seeds[1]], nonseeds_idx[nonseeds_to_seeds[0]]])

In [ ]:
# Visualize this!
plt.figure(figsize=(10, 10))
seeds_df = event_df[np.isin(event_df.hit_number.values, seeds_idx.long().numpy())]
nonseeds_df = event_df[np.isin(event_df.hit_number.values, nonseeds_idx.long().numpy())]
plt.scatter(nonseeds_df.posx, nonseeds_df.posy, marker='o', c='b', s=3)
plt.plot(event_df.posx.values[nonseeds_to_seeds], event_df.posy.values[nonseeds_to_seeds], c="g", alpha=0.3)
plt.plot(event_df.posx.values[true_edges], event_df.posy.values[true_edges], c="m", alpha=0.2)
plt.scatter(seeds_df.posx, seeds_df.posy, marker='o', c='k', s=100)

In [ ]:
combined_graph = torch.cat([nonseeds_to_seeds, true_edges], dim=-1)

In [ ]:
sparse_edges = to_scipy_sparse_matrix(combined_graph, num_nodes = len(event_df))

In [ ]:
_, candidate_labels = sps.csgraph.connected_components(sparse_edges, directed=False, return_labels=True)  
labels = torch.from_numpy(candidate_labels).long()

In [ ]:
np.mean([len(event.edge_index.unique()) for event in tqdm(model.trainset)])

In [ ]:
input_dir = "/global/cfs/cdirs/m3443/data/PowerWeek/train/train"
num_events = sum(model.hparams["data_split"])
csv_files = sorted([os.path.join(input_dir, f) for f in os.listdir(input_dir) if f.endswith('.csv')])[:num_events//1000 + 1]
events_df = pd.concat([pd.read_csv(f) for f in sorted(csv_files)])
if num_events is not None:
    events_df = events_df[events_df["event"].isin(sorted(events_df["event"].unique())[:num_events])]
events_df['clusterID'] = events_df['clusterID'].astype(np.uint64) # Needed for some reason?

In [ ]:
pd.options.mode.chained_assignment = None
def label_hits(event, events_df, max_dist=None):
    true_edges = event.edge_index[:, event.y.bool()]
    
    seeds_idx = true_edges.unique()
    
    event_df = events_df[events_df.event == event.event_id]
    
    # Collect nonseeds in another tensor
    nonseeds_idx = torch.from_numpy(event_df.hit_number[~np.isin(event_df.hit_number.values, seeds_idx.long().numpy())].values).unique()

    # For each nonseed find closest seed with knn=1
    nonseeds_to_seeds = knn(torch.from_numpy(event_df[np.isin(event_df.hit_number.values, seeds_idx.long().numpy())][['posx', 'posy']].to_numpy()), torch.from_numpy(event_df[np.isin(event_df.hit_number.values, nonseeds_idx.long().numpy())][['posx', 'posy']].to_numpy()), 1)

    # Convert 0, .., N indices back to original seed_idx and nonseed_idx
    nonseeds_to_seeds = torch.stack([seeds_idx[nonseeds_to_seeds[1]], nonseeds_idx[nonseeds_to_seeds[0]]])
    
    if max_dist is not None:
        positions = torch.from_numpy(events_df[["posx", "posy", "posz"]].values)
        nonseeds_to_seeds = nonseeds_to_seeds[:, torch.sqrt(torch.sum((positions[nonseeds_to_seeds[0]] - positions[nonseeds_to_seeds[1]])**2, dim=-1)) < max_dist]
    
    combined_graph = torch.cat([nonseeds_to_seeds, true_edges], dim=-1)
    sparse_edges = to_scipy_sparse_matrix(combined_graph, num_nodes = len(event_df))
    _, candidate_labels = sps.csgraph.connected_components(sparse_edges, directed=False, return_labels=True)  
    labels = torch.from_numpy(candidate_labels).long()
    
    event_df['tmp_clusterID'] = labels

    # encode the labels to make sure it's unique across all events 
    str_ids = event_df['event'].astype('str') + "_" + event_df['tmp_clusterID'].astype('str')
    event_df['labelID'] = [xxhash.xxh64_intdigest(x, seed=0) for x in str_ids.values]
    
    return event_df

In [ ]:
labelled_events_df = []
for event in tqdm(model.trainset[:2000]):
    try:
        labelled_events_df.append(label_hits(event, events_df))
    except:
        pass
labelled_events_df = pd.concat(labelled_events_df)
print(f"Vscore: {weighted_v_score(labels_true=labelled_events_df['clusterID'], labels_pred=labelled_events_df['labelID'], labels_weight=labelled_events_df['E'])[2]}")

In [ ]:
print(f"Vscore: {weighted_v_score(labels_true=labelled_events_df['clusterID'], labels_pred=labelled_events_df['labelID'], labels_weight=labelled_events_df['E'])[2]}")

In [ ]:
worst_performance = 0
worst_score = None
for event in tqdm(model.trainset[:100]):
    try:
        labelled_event = label_hits(event, events_df)
    except:
        pass
    score = weighted_v_score(labels_true=labelled_event['clusterID'], labels_pred=labelled_event['labelID'], labels_weight=labelled_event['E'])[2]
    performance = labelled_event.E.sum() / score
    if performance > worst_performance:
        print(performance)
        print(event)
        print(score)
        worst_performance = performance
        worst_score = score

In [ ]:
hard_event = model.trainset[61]
hard_event_df = events_df[events_df.event == hard_event.event_id]

In [ ]:
hard_event

In [ ]:
hard_event_df.E.sum() / worst_score

In [ ]:
true_edges = hard_event.edge_index[:, hard_event.y.bool()]

In [ ]:
# Collect seeds in one tensor
seeds_idx = true_edges.unique()

In [ ]:
# Collect nonseeds in another tensor
nonseeds_idx = torch.from_numpy(hard_event_df.hit_number[~np.isin(hard_event_df.hit_number.values, seeds_idx.long().numpy())].values).unique()

In [ ]:
# For each nonseed find closest seed with knn=1
nonseeds_to_seeds = knn(torch.from_numpy(hard_event_df[np.isin(hard_event_df.hit_number.values, seeds_idx.long().numpy())][['posx', 'posy']].to_numpy()), torch.from_numpy(hard_event_df[np.isin(hard_event_df.hit_number.values, nonseeds_idx.long().numpy())][['posx', 'posy']].to_numpy()), 1)

# Convert 0, .., N indices back to original seed_idx and nonseed_idx
nonseeds_to_seeds = torch.stack([seeds_idx[nonseeds_to_seeds[1]], nonseeds_idx[nonseeds_to_seeds[0]]])

In [ ]:
cluster_id = hard_event_df.clusterID.values

In [ ]:
true_neighbor_edges = nonseeds_to_seeds[:, cluster_id[nonseeds_to_seeds[0]] == cluster_id[nonseeds_to_seeds[1]]]
fake_neighbor_edges = nonseeds_to_seeds[:, cluster_id[nonseeds_to_seeds[0]] != cluster_id[nonseeds_to_seeds[1]]]

In [ ]:
# Visualize this!
plt.figure(figsize=(10, 10))
seeds_df = hard_event_df[np.isin(hard_event_df.hit_number.values, seeds_idx.long().numpy())]
nonseeds_df = hard_event_df[np.isin(hard_event_df.hit_number.values, nonseeds_idx.long().numpy())]
plt.scatter(nonseeds_df.posx, nonseeds_df.posy, marker='o', c='b', s=3)
plt.plot(hard_event_df.posx.values[nonseeds_to_seeds], hard_event_df.posy.values[nonseeds_to_seeds], c="g", alpha=0.3)
plt.plot(hard_event_df.posx.values[true_edges], hard_event_df.posy.values[true_edges], c="m", alpha=0.2)
plt.scatter(seeds_df.posx, seeds_df.posy, marker='o', c='k', s=100)

In [ ]:
# Visualize this!
plt.figure(figsize=(10, 10))
seeds_df = hard_event_df[np.isin(hard_event_df.hit_number.values, seeds_idx.long().numpy())]
nonseeds_df = hard_event_df[np.isin(hard_event_df.hit_number.values, nonseeds_idx.long().numpy())]
plt.plot(hard_event_df.posx.values[true_neighbor_edges], hard_event_df.posy.values[true_neighbor_edges], c="g", alpha=0.2)
plt.plot(hard_event_df.posx.values[fake_neighbor_edges], hard_event_df.posy.values[fake_neighbor_edges], c="r", alpha=0.2)
plt.plot(hard_event_df.posx.values[true_edges], hard_event_df.posy.values[true_edges], c="m", alpha=0.2)
plt.scatter(seeds_df.posx, seeds_df.posy, marker='o', c=seeds_df.clusterID, s=100)
plt.scatter(nonseeds_df.posx, nonseeds_df.posy, marker='o', c=nonseeds_df.clusterID, s=15)

In [ ]:
# Visualize this!
plt.figure(figsize=(10, 10))
seeds_df = hard_event_df[np.isin(hard_event_df.hit_number.values, seeds_idx.long().numpy())]
nonseeds_df = hard_event_df[np.isin(hard_event_df.hit_number.values, nonseeds_idx.long().numpy())]
plt.scatter(nonseeds_df.posx, nonseeds_df.posy, marker='o', c='b', s=3)
plt.plot(hard_event_df.posx.values[fake_neighbor_edges], hard_event_df.posy.values[fake_neighbor_edges], c="r", alpha=0.5)
plt.plot(hard_event_df.posx.values[true_neighbor_edges], hard_event_df.posy.values[true_neighbor_edges], c="g", alpha=0.5)
plt.plot(hard_event_df.posx.values[true_edges], hard_event_df.posy.values[true_edges], c="m", alpha=0.2)
plt.scatter(seeds_df.posx, seeds_df.posy, marker='o', c='k', s=100)

In [ ]:
scores, energies, num_seeds = [], [], []
for event in tqdm(model.trainset[:1000]):
    try:
        labelled_event = label_hits(event, events_df)
    except:
        pass
    scores.append(weighted_v_score(labels_true=labelled_event['clusterID'], labels_pred=labelled_event['labelID'], labels_weight=labelled_event['E'])[2])
    energies.append(labelled_event.E.sum())
    num_seeds.append(len(event.edge_index.unique()))

In [ ]:
sns.histplot(np.array(energies) / np.array(scores))

In [ ]:
sns.histplot(scores)

In [ ]:
np.mean(scores)

In [ ]:
sns.histplot(x=scores, y=energies)

In [ ]:
sns.histplot(x=scores, y=num_seeds)

In [ ]:
sns.histplot(x=energies, y=num_seeds)

## 2. Inference!

In [ ]:
checkpoint_file = "/global/cfs/cdirs/m3443/data/PowerWeek/checkpoints/classifier.ckpt"

In [ ]:
model = MemberClassification.load_from_checkpoint(checkpoint_file)

In [ ]:
model.hparams["data_split"] = [5000, 100, 100]

In [ ]:
model.setup(stage="fit")

In [ ]:
input_dir = "/global/cfs/cdirs/m3443/data/PowerWeek/train/train"
num_events = sum(model.hparams["data_split"])
csv_files = sorted([os.path.join(input_dir, f) for f in os.listdir(input_dir) if f.endswith('.csv')])[:num_events//1000 + 1]
events_df = pd.concat([pd.read_csv(f) for f in sorted(csv_files)])
if num_events is not None:
    events_df = events_df[events_df["event"].isin(sorted(events_df["event"].unique())[:num_events])]
events_df['clusterID'] = events_df['clusterID'].astype(np.uint64) # Needed for some reason?

In [ ]:
pd.options.mode.chained_assignment = None
def label_hits(event, events_df, pred_edges, max_dist=None):
    
    seeds_idx = pred_edges.unique()
    
    event_df = events_df[events_df.event == event.event_id]
    
    # Collect nonseeds in another tensor
    nonseeds_idx = torch.from_numpy(event_df.hit_number[~np.isin(event_df.hit_number.values, seeds_idx.long().numpy())].values).unique()

    # For each nonseed find closest seed with knn=1
    nonseeds_to_seeds = knn(torch.from_numpy(event_df[np.isin(event_df.hit_number.values, seeds_idx.long().numpy())][['posx', 'posy']].to_numpy()), torch.from_numpy(event_df[np.isin(event_df.hit_number.values, nonseeds_idx.long().numpy())][['posx', 'posy']].to_numpy()), 1)

    # Convert 0, .., N indices back to original seed_idx and nonseed_idx
    nonseeds_to_seeds = torch.stack([seeds_idx[nonseeds_to_seeds[1]], nonseeds_idx[nonseeds_to_seeds[0]]])
    
    if max_dist is not None:
        positions = torch.from_numpy(events_df[["posx", "posy", "posz"]].values)
        nonseeds_to_seeds = nonseeds_to_seeds[:, torch.sqrt(torch.sum((positions[nonseeds_to_seeds[0]] - positions[nonseeds_to_seeds[1]])**2, dim=-1)) < max_dist]
    
    # Add the seed-seed edges and the seed-nonseed edges into the same graph
    combined_graph = torch.cat([nonseeds_to_seeds, pred_edges], dim=-1)
    sparse_edges = to_scipy_sparse_matrix(combined_graph, num_nodes = len(event_df))
    
    # Perform a connected components algorithm on the graph
    _, candidate_labels = sps.csgraph.connected_components(sparse_edges, directed=False, return_labels=True)  
    labels = torch.from_numpy(candidate_labels).long()
    
    event_df['tmp_clusterID'] = labels

    # encode the labels to make sure it's unique across all events 
    str_ids = event_df['event'].astype('str') + "_" + event_df['tmp_clusterID'].astype('str')
    event_df['labelID'] = [xxhash.xxh64_intdigest(x, seed=0) for x in str_ids.values]
    
    return event_df

Let's test on the training data first

In [ ]:
labelled_events_df = []
for event in tqdm(model.trainset):
    try:
        with torch.no_grad():
            edge_scores = model.cuda()(event.x.cuda()).cpu().squeeze()
        labelled_events_df.append(label_hits(event, events_df, event.edge_index[:, edge_scores > 0.6]))
    except:
        pass
labelled_events_df = pd.concat(labelled_events_df)
print(f"Vscore: {weighted_v_score(labels_true=labelled_events_df['clusterID'], labels_pred=labelled_events_df['labelID'], labels_weight=labelled_events_df['E'])[2]}")

### Test Dataset

Now, to build the test set

In [ ]:
checkpoint_file = "/global/cfs/cdirs/m3443/data/PowerWeek/checkpoints/classifier.ckpt"

In [ ]:
model = MemberClassification.load_from_checkpoint(checkpoint_file)

In [ ]:
model.hparams["data_split"] = [10000, 0, 0]
model.hparams["input_dir"] = "/global/cfs/cdirs/m3443/data/PowerWeek/test/test"

In [ ]:
model.setup(stage="fit")

In [ ]:
input_dir = "/global/cfs/cdirs/m3443/data/PowerWeek/test/test"
num_events = sum(model.hparams["data_split"])
csv_files = sorted([os.path.join(input_dir, f) for f in os.listdir(input_dir) if f.endswith('.csv')])[:num_events//1000 + 1]
events_df = pd.concat([pd.read_csv(f) for f in tqdm(sorted(csv_files))])
if num_events is not None:
    events_df = events_df[events_df["event"].isin(sorted(events_df["event"].unique())[:num_events])]

In [ ]:
labelled_events_df = []
for event in tqdm(model.trainset):
    try:
        with torch.no_grad():
            edge_scores = model.cuda()(event.x.cuda()).cpu().squeeze()
        labelled_events_df.append(label_hits(event, events_df, event.edge_index[:, edge_scores > 0.65]))
    except:
        print(f"Error with event {event}")
labelled_events_df = pd.concat(labelled_events_df)

There are some missing rows for some reason! Let's just add them back in with random labels...

In [ ]:
missing_rows = events_df[~events_df.uniqueID.isin(labelled_events_df.uniqueID)]

In [ ]:
missing_rows['labelID'] = np.random.randint(0, 1000000, (len(missing_rows)))

In [ ]:
labelled_events_df = pd.concat([labelled_events_df, missing_rows])

Save the data

In [ ]:
labelled_events_df["clusterID"] = labelled_events_df["labelID"]

In [ ]:
labelled_events_df[["uniqueID", "clusterID"]].to_parquet("membership_classification.parquet")